# Your mission

You started working on the Ecowatt project at RTE. In order to avoid possible shortage, one must plan for peaks in national electricity. You manager Mark is going on holidays for a week. You will be sole responsible for forecasting the weekly demand, while he is absent.

In order to prevent electricity shortage, you must accurately forecast the demand 7 days ahead, on an hourly basis.

Your mission is to train an accurate predictive model with the lowest root mean squared error (RMSE). Mark is a very technical guy, he likes to understand all technical details and would like you to compare the performances of classical models and neural-net based models.


Your **target variable** is the consommation_totale

**Data source** : https://data.enedis.fr/pages/accueil/

# Import

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/MyDrive/Colab Notebooks/EI_ST4_G1/EI_TS_CS-20230526T084435Z-001/EI_TS_CS")

Mounted at /content/gdrive


In [ ]:
%run ./utils.ipynb

In [ ]:
FILE_PATH = "data/bilan.csv"
TARGET = "consommation_totale"
EXOGENEOUS= "Température normale lissée (°C)"

## Prepare the data

Define here the range of your train/test split

In [ ]:
df = pd.read_csv(FILE_PATH)

X_train = df[-1000:-100]
X_test = df[-100:]

In [ ]:
df

,Unnamed: 0,horodate,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),consommation_totale,Consommation totale télérelevée (W),Consommation HTA télérelevée (W),Consommation totale profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°C),Température normale lissée (°C),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
0,0.0,2019-07-03T22:00:00+02:00,7.0,3.383306e+10,2.725795e+09,2.286131e+09,3.475336e+10,1.174429e+10,1.168837e+10,2.300908e+10,...,2.154923e+07,1.414142e+07,1102801.0,22.9,20.6,5.076127e+09,1.471273e+07,67.0,1.323963e+10,241976206.0
1,0.0,2019-07-04T00:30:00+02:00,7.0,3.010544e+10,2.969232e+09,2.079282e+09,3.126325e+10,1.114404e+10,1.109063e+10,2.011922e+10,...,6.961098e+06,0.000000e+00,762042.0,22.2,20.3,5.328515e+09,4.912830e+05,73.0,1.252451e+10,223556213.0
2,0.0,2019-07-04T03:30:00+02:00,7.0,2.549278e+10,2.719259e+09,1.798193e+09,2.652442e+10,1.081601e+10,1.075762e+10,1.570841e+10,...,6.813437e+06,0.000000e+00,762042.0,21.9,20.0,4.603394e+09,4.877070e+05,83.0,1.213424e+10,164460823.0
3,0.0,2019-07-04T05:00:00+02:00,7.0,2.621578e+10,2.319118e+09,1.812301e+09,2.718336e+10,1.163813e+10,1.158143e+10,1.554523e+10,...,6.622984e+06,0.000000e+00,762042.0,21.7,19.8,4.169759e+09,4.905970e+05,89.0,1.310457e+10,180465603.0
4,0.0,2019-07-04T06:30:00+02:00,7.0,2.935876e+10,1.855103e+09,1.956309e+09,3.027574e+10,1.296523e+10,1.290623e+10,1.731052e+10,...,5.208770e+07,4.553791e+07,762042.0,21.5,19.7,3.727644e+09,9.889918e+07,92.0,1.444158e+10,215049347.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87644,0.0,2021-03-19T10:30:00+01:00,3.0,4.738432e+10,4.935468e+09,4.457043e+09,5.241930e+10,1.926923e+10,1.592864e+10,3.315006e+10,...,1.018214e+09,1.010288e+09,2205471.0,6.7,8.7,8.539976e+09,3.006684e+09,29.0,1.633513e+10,375074101.0
87645,0.0,2021-03-19T12:30:00+01:00,3.0,4.715819e+10,5.850612e+09,4.614302e+09,5.291407e+10,1.780583e+10,1.476675e+10,3.510824e+10,...,1.417184e+09,1.409243e+09,2205471.0,7.3,9.7,9.371482e+09,3.893055e+09,39.0,1.511712e+10,329267532.0
87646,0.0,2021-03-19T13:00:00+01:00,3.0,4.595804e+10,6.111693e+09,4.507434e+09,5.174172e+10,1.772636e+10,1.472480e+10,3.401536e+10,...,1.444703e+09,1.436761e+09,2205471.0,7.4,10.0,9.426723e+09,4.015956e+09,42.0,1.507785e+10,317486066.0
87647,0.0,2021-03-19T13:30:00+01:00,3.0,4.468497e+10,6.136824e+09,4.373983e+09,5.054535e+10,1.773296e+10,1.477001e+10,3.281239e+10,...,1.439957e+09,1.432042e+09,2205471.0,7.5,10.0,9.435040e+09,3.969226e+09,42.0,1.512530e+10,314407289.0


# Modeling with ARIMAX


## Modeling
The following code allows ARIMAX modelling using the temperature as an exogeneous variable.

In [ ]:
order = (2,0,2)
seasonal_order = (2, 1, 2, 12)

errors, predictions = evaluate_sarimax_model(
    X_train[TARGET],
    X_test[TARGET],
    X_train[EXOGENEOUS],
    X_test[EXOGENEOUS],
    order,
    seasonal_order
    )
errors

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood op

1.5296999593220418e+19

## Search for the best ARIMA model
We use grid search to search for the best ARIMA parameters that gives the lowest error. This follows the Box-Jenkins methology.

In [10]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

def sarimax_grid_search(X_train, X_test, exog_train, exog_test, p_values, d_values, q_values, P_values, D_values, Q_values, S_values):
    """
    Grid search of SARIMAX model parameters

    Arguments:
    ----------
    - X_train: numpy array
        - train set
    - X_test: numpy array
        - test set
    - exog_train: numpy array
        - exogenous variable for training set
    - exog_test: numpy array
        - exogenous variable for test set
    - p_values: numpy array
        - a list of values for p parameter
    - d_values: numpy array
        - a list of values for d parameter
    - q_values: numpy array
        - a list of values for q parameter
    - P_values: numpy array
        - a list of values for seasonal P parameter
    - D_values: numpy array
        - a list of values for seasonal D parameter
    - Q_values: numpy array
        - a list of values for seasonal Q parameter
    - S_values: numpy array
        - a list of values for seasonal period S parameter

    Returns:
    --------
    - best_cfg: tuple
        - best model parameter (p, d, q, P, D, Q, S)
    - best_score: float
        - best RMSE score
    """

    best_score, best_cfg = float("inf"), None

    for p in p_values:
        for d in d_values:
            for q in q_values:
                for P in P_values:
                    for D in D_values:
                        for Q in Q_values:
                            for S in S_values:
                                order = (p, d, q)
                                seasonal_order = (P, D, Q, S)
                                try:
                                    rmse, _ = evaluate_sarimax_model(X_train, X_test, exog_train, exog_test, order, seasonal_order)
                                    if rmse < best_score:
                                        best_score, best_cfg = rmse, (p, d, q, P, D, Q, S)
                                    print("SARIMAX(%d,%d,%d)x(%d,%d,%d,%d) RMSE=%.3f" % (p, d, q, P, D, Q, S, rmse))

                                except:
                                    continue



In [11]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
best_cfg, best_score = sarimax_grid_search(X_train[TARGET],
                                            X_test[TARGET],
                                         X_train[EXOGENEOUS],
                                         X_test[EXOGENEOUS],
                                            range(1,2),range(0,2),range(0,2),range(0,2),range(0,2),range(0,2),range(0,2))

SARIMAX(1,0,0)x(0,0,0,0) RMSE=46846690431330910208.000
SARIMAX(1,0,1)x(0,0,0,0) RMSE=36948834408288010240.000
SARIMAX(1,1,0)x(0,0,0,0) RMSE=16132245735575054336.000
SARIMAX(1,1,1)x(0,0,0,0) RMSE=13531751644737028096.000


TypeError: ignored

In [ ]:
print(best_cfg, best_score)

## Visualization
To have a better view on the difference between true and predict values, we visualize them by plotting both the signals.

In [ ]:
# prepare the dataset for plotting
predict_date = df["horodate"]
df_predict = pd.DataFrame(zip(predict_date[-100:],
                              predictions, X_test[TARGET].values),
                          columns=["date", "predict", "true"])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_predict["date"], y=df_predict["predict"], name="predict"))
fig.add_trace(go.Scatter(x=df_predict["date"], y=df_predict["true"], name="true"))

fig.update_layout(title="Predictions vs true values")

# Modeling with other models

Try other other models : random forest, xgboost ...